In [1]:
%%bash
mkdir dataset
curl https://storage.googleapis.com/deepmind-media/research/Kinetics_700.zip --output dataset/kinects_700.zip

mkdir: não foi possível criar o diretório “dataset”: Arquivo existe
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.7M  100 21.7M    0     0  12.9M      0  0:00:01  0:00:01 --:--:-- 12.9M


In [2]:
%%bash
cd dataset
unzip kinects_700.zip

Archive:  kinects_700.zip


replace kinetics_700_readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [3]:
%%bash
cd dataset
unzip kinetics_700_train.zip
unzip kinetics_700_test.zip
unzip kinetics_700_val.zip

Archive:  kinetics_700_train.zip


replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [unzip kin]
replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [etics_700]
replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [_test.zip]
replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [{ENTER}]
replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [unzip kin]
replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [etics_700]
replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [_val.zip]
replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [4]:
import pandas as pd
import argparse
import os
import shutil
import subprocess
import pytube
from joblib import delayed
from joblib import Parallel

In [5]:
REQUIRED_COLUMNS = ['label', 'youtube_id', 'time_start', 'time_end', 'split', 'is_cc']
TRIM_FORMAT = '%06d'
URL_BASE = 'https://www.youtube.com/watch?v='

VIDEO_EXTENSION = '.mp4'
VIDEO_FORMAT = 'mp4'
TOTAL_VIDEOS = 0

In [6]:
BASE_PATH = os.path.join('/', 'media' ,'renato' ,'DATA','kinects')

TRAIN_VIDEOS_PATH = os.path.join(BASE_PATH, 'train')
VALIDATE_VIDEOS_PATH = os.path.join(BASE_PATH, 'validate')
TEST_VIDEOS_PATH = os.path.join(BASE_PATH, 'test')

In [7]:
if not os.path.exists(BASE_PATH):
    os.mkdir(BASE_PATH)
else:
    print(f'Path {BASE_PATH} already exists!')

Path /media/renato/DATA/kinects already exists!


In [10]:
if not os.path.exists(TRAIN_VIDEOS_PATH):
    os.mkdir(TRAIN_VIDEOS_PATH)
else:    
    print(f'Path to dataset {TRAIN_VIDEOS_PATH} already exists!')
    
if not os.path.exists(VALIDATE_VIDEOS_PATH):
    os.mkdir(VALIDATE_VIDEOS_PATH)
else:    
    print(f'Path to dataset {VALIDATE_VIDEOS_PATH} already exists!')
    
if not os.path.exists(TEST_VIDEOS_PATH):
    os.mkdir(TEST_VIDEOS_PATH)
else:    
    print(f'Path to dataset {TEST_VIDEOS_PATH} already exists!')

Path to dataset /media/renato/DATA/kinects/train already exists!
Path to dataset /media/renato/DATA/kinects/validate already exists!
Path to dataset /media/renato/DATA/kinects/test already exists!


In [11]:
def create_file_structure(path, folders_names):
    mapping = {}
    if not os.path.exists(path):
        os.mkdir(path)
    for name in folders_names:
        dir_ = os.path.join(path, name)
        if not os.path.exists(dir_):
            os.mkdir(dir_)
        mapping[name] = dir_
    return mapping

In [18]:
def download_clip(row, label_to_dir, trim, count):
    label = row['label']
    filename = row['youtube_id']
    time_start = row['time_start']
    time_end = row['time_end']

    output_path = label_to_dir['tmp'] if trim else label_to_dir[label]

    if not os.path.exists(os.path.join(output_path, filename + VIDEO_EXTENSION)):
        print('Start downloading: ', filename)
        try:
            pytube.YouTube(URL_BASE + filename).\
                streams.filter(subtype=VIDEO_FORMAT).first().\
                download(output_path, filename)
            print('Finish downloading: ', filename)
        except KeyError as e:
            print(f'Unavailable video {e}')
            return
        except Exception as e:
            print(f'Don\'t know why something went wrong: {e}')
            return
    else:
        print('Already downloaded: ', filename)

    if trim:
        start = str(time_start)
        end = str(time_end - time_start)

        input_filename = os.path.join(output_path, filename + VIDEO_EXTENSION)
        output_filename = os.path.join(label_to_dir[label],
                                       filename + '_{}_{}'.format(start, end) + VIDEO_EXTENSION)

        if os.path.exists(output_filename):
            print('Already trimmed: ', filename)
        else:
            print('Start trimming: ', filename)
            command = 'ffmpeg -i "{input_filename}" ' \
                      '-ss {time_start} ' \
                      '-t {time_end} ' \
                      '-c:v libx264 -c:a copy -threads 1 ' \
                      '"{output_filename}"'.format(
                           input_filename=input_filename,
                           time_start=start,
                           time_end=end,
                           output_filename=output_filename
                       )
            try:
                subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
            except subprocess.CalledProcessError as e:
                print(f'Error {e} while trimming: ', filename)
                return False
            print('Finish trimming: ', filename)

    print('Processed %i out of %i' % (count + 1, TOTAL_VIDEOS))

In [19]:
def main(input_csv, output_dir, trim, num_jobs):
    global TOTAL_VIDEOS

    assert input_csv[-4:] == '.csv', 'Provided input is not a .csv file'
    links_df = pd.read_csv(input_csv)
    assert all(elem in REQUIRED_COLUMNS for elem in links_df.columns.values),\
        'Input csv doesn\'t contain required columns.'

    folders_names = links_df['label'].unique().tolist() + ['tmp']
    label_to_dir = create_file_structure(path=output_dir,
                                         folders_names=folders_names)

    TOTAL_VIDEOS = links_df.shape[0]
    Parallel(n_jobs=num_jobs)(delayed(download_clip)(
            row, label_to_dir, trim, count) for count, row in links_df.iterrows())

    shutil.rmtree(label_to_dir['tmp'])

In [20]:
main('dataset/kinetics_700_val.csv', VALIDATE_VIDEOS_PATH, True, 60)

KeyboardInterrupt: 

In [26]:
main('dataset/kinetics_700_train.csv', TRAIN_VIDEOS_PATH, True, 60)

KeyboardInterrupt: 

In [ ]:
main('dataset/kinetics_700_test.csv', test_VIDEOS_PATH, True, 60)